In [1]:
!javac de/unima/ki/anyburl/*.java -d build

In [2]:
!jar cfv AnyBURL-23-1.jar -C build .

added manifest
adding: de/(in = 0) (out= 0)(stored 0%)
adding: de/unima/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/anyburl/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/anyburl/Apply.class(in = 9153) (out= 5041)(deflated 44%)
adding: de/unima/ki/anyburl/Eval.class(in = 4680) (out= 2438)(deflated 47%)
adding: de/unima/ki/anyburl/exceptions/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/anyburl/exceptions/RuleFunctionalityBasicSupportOnly.class(in = 567) (out= 384)(deflated 32%)
adding: de/unima/ki/anyburl/threads/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/anyburl/threads/RuleWriterAsThread.class(in = 4034) (out= 2036)(deflated 49%)
adding: de/unima/ki/anyburl/threads/Predictor.class(in = 1523) (out= 657)(deflated 56%)
adding: de/unima/ki/anyburl/threads/Scorer.class(in = 4873) (out= 2552)(deflated 47%)
adding: de/unima/ki/anyburl/eval/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/anyburl/eval/Candidate.class(in = 935) 

In [3]:
import pandas as pd
import zipfile
import os
from tqdm import tqdm

In [4]:
os.cpu_count()

32

In [5]:
import psutil
psutil.virtual_memory()

svmem(total=99980713984, available=96070742016, percent=3.9, used=3447033856, free=96290889728, active=604057600, inactive=220254208, buffers=11599872, cached=231190528, shared=2727936, slab=274337792)

In [6]:
def train(trace_id):
    train_prop = f'PATH_TRAINING = MSCallGraph_traces/train/{trace_id}_transductive_train.tsv\nPATH_OUTPUT   = rules\nSNAPSHOTS_AT = 10\nWORKER_THREADS = 30\nSAFE_PREFIX_MODE = true'
    f = open("train_prop.txt", "w")
    f.write(train_prop)
    f.close()
    _=!java -Xmx72G -cp AnyBURL-23-1.jar de.unima.ki.anyburl.Learn train_prop.txt

In [7]:
def predict(trace_id):
    predict_prop = f'PATH_TRAINING = MSCallGraph_traces/train/{trace_id}_transductive_train.tsv\nPATH_VALID    = MSCallGraph_traces/train/{trace_id}_transductive_train.tsv\nPATH_TEST     = MSCallGraph_traces/test/{trace_id}_transductive_test.tsv\nPATH_RULES    = rules-10\nPATH_OUTPUT   = preds-10\nWORKER_THREADS = 30\nTOP_K_OUTPUT = 100\nSAFE_PREFIX_MODE = true'
    f = open("predict_prop.txt", "w")
    f.write(predict_prop)
    f.close()
    _=!java -Xmx72G -cp AnyBURL-23-1.jar de.unima.ki.anyburl.Apply predict_prop.txt

In [8]:
def test(trace_id):
    eval_prop = f'PATH_TRAINING = MSCallGraph_traces/train/{trace_id}_transductive_train.tsv\nPATH_VALID    = MSCallGraph_traces/train/{trace_id}_transductive_train.tsv\nPATH_TEST     = MSCallGraph_traces/test/{trace_id}_transductive_test.tsv\nPATH_PREDICTIONS = preds-10\nTOP_K = 100\nSAFE_PREFIX_MODE = true'
    f = open("eval_prop.txt", "w")
    f.write(eval_prop)
    f.close()
    to_return = !java -Xmx72G -cp AnyBURL-23-1.jar de.unima.ki.anyburl.Eval eval_prop.txt
    return to_return[-1]

In [9]:
def test_metrics(traceid):
    train(traceid)
    predict(traceid)
    return(test(traceid))

In [10]:
def test_on_traces(model_name):
    directory = model_name+"_testing_traces"

    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f'Directory {directory} created successfully!')
    else:
        print(f'Directory {directory} already exists.')
        


    # Specifica il percorso della cartella da cui si vogliono ottenere i nomi dei file
    folder_path =  "MSCallGraph_traces/train/"

    files_list = []
    for filename in os.listdir(folder_path):
        # Ottenere il nome del file
        file_name = os.path.basename(filename)
        files_list.append(file_name)
    
    metric_names = ['hits_at_1','hits_at_3','hits_at_10','mrr']
    all_traces_list = []
    # Scansione di ogni file nella cartella
    with tqdm(desc=f'{model_name} testing traces', total=len(files_list)) as progress_bar:
        for file_name in files_list:
            # Stampa il nome del file
            all_traces_list.append(test_metrics(file_name[:-23]).split())
            progress_bar.update(1)


    trace_df = pd.DataFrame(all_traces_list,columns=metric_names)
    trace_df.to_csv(f"{model_name}_testing_traces/trace_test.csv")

    
    display(trace_df)
    def zip_folder(folder_path, output_path):
        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    zipf.write(os.path.join(root, file))

    folder_path = model_name+"_testing_traces"
    output_path = f'{folder_path}.zip'

    zip_folder(folder_path, output_path)

In [11]:
test_on_traces("anyburl")

Directory anyburl_testing_traces already exists.


anyburl testing traces: 100%|█████████████████████████████████████████████████████| 1027/1027 [4:16:12<00:00, 14.97s/it]


,hits_at_1,hits_at_3,hits_at_10,mrr
0,0.0000,0.1667,0.3333,0.1111
1,0.5000,1.0000,1.0000,0.7500
2,0.0000,0.0000,0.0000,0.0000
3,0.0000,0.0000,0.0000,0.0000
4,0.2000,0.2000,0.2000,0.2000
...,...,...,...,...
1022,0.0000,0.0000,1.0000,0.1833
1023,0.0000,1.0000,1.0000,0.5000
1024,0.2143,0.2857,0.5714,0.2851
1025,0.0000,0.0000,0.0000,0.0000
